#### Libraries

In [1]:
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from catboost import CatBoostRegressor, Pool
import pickle

pd.set_option('display.max_columns', 500)

#### Load Data

In [2]:
data_dir = '/Users/konstantinivanov/Documents/work/PIK/files-pik_digital_day_final'
input_dir = '/Users/konstantinivanov/Documents/work/PIK/input'
output_dir = '/Users/konstantinivanov/Documents/work/PIK/output'
sub_dir = '/Users/konstantinivanov/Documents/work/PIK/sub'

#### Prepare Data

In [4]:
train_data = pd.read_csv(data_dir + '/train.csv', encoding='cp1251')
test_data = pd.read_csv(data_dir + '/test.csv', encoding='cp1251')
test_sub = pd.read_csv(sub_dir + '/sub_plusmay.csv', encoding='cp1251')

In [5]:
train_data.shape

(9244, 56)

In [6]:
test_data.shape

(1817, 48)

In [7]:
train_data.drop(['start_square', 'plan_s', 'plan_m', 'plan_l', 'vid_0', 'vid_1', 'vid_2'], axis=1, inplace=True)
train_data['fl_mod'] = 0
train_data.to_csv(input_dir + "/train_full_mod.csv", index=False)

test_data['value'] = test_sub['value']
test_data['fl_mod'] = 1
test_data[test_data['date1']=='2018-04-01']['fl_mod'] = 2
test_data[test_data['date1']=='2018-05-01']['fl_mod'] = 3
test_data.to_csv(input_dir + "/test_full_mod.csv", index=False, columns = train_data.columns)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
train_data.shape

(9244, 50)

In [8]:
test_data.shape

(1817, 50)

#### Validation

In [3]:
TEST_FILE = input_dir + '/test_full_mod.csv' #add
CD_FILE = input_dir + '/train_mod.txt'

train_data = pd.read_csv(input_dir + "/train_full_mod.csv")
test_data = pd.read_csv(TEST_FILE)
train_data = pd.concat([train_data, test_data])
train_data = train_data.reset_index().drop(['index'], axis=1)

In [16]:
K = 4

kf = KFold(n_splits=K, random_state=42, shuffle=True)
kf.get_n_splits(range(len(train_data)))

print(kf)  

for i, (train_index, valid_index) in enumerate(kf.split(range(len(train_data)))):
    train = train_data.loc[train_index]
    valid = train_data.loc[valid_index]

    train.to_csv(input_dir + f"/train_{i}.csv",index=False)
    valid.to_csv(input_dir + f"/valid_{i}.csv",index=False)

KFold(n_splits=4, random_state=42, shuffle=True)


#### Fit and Predict

In [4]:
test_pool = Pool(TEST_FILE, column_description=CD_FILE, has_header=True, delimiter=",")

In [5]:
train_data.shape

(11061, 50)

In [6]:
test_data.shape

(1817, 50)

In [17]:
iterations = 5000
learning_rate = 0.05
depth = 6
random_seed = 42

Regressors = [
    CatBoostRegressor(iterations=iterations, learning_rate=learning_rate, depth=depth, random_seed=random_seed)
]

In [18]:
K = 4
def FitOnKFolds(model):
    cum_y_test = np.zeros((len(test_data), 1))
    pred_y_train = np.zeros((len(train_data), 1))
    
    for i in range(K):
        TRAIN_FILE = input_dir + f'/train_{i}.csv'
        VAL_FILE = input_dir + f'/valid_{i}.csv'
        
        train_pool = Pool(TRAIN_FILE, column_description=CD_FILE, has_header=True, delimiter=",")
        val_pool = Pool(VAL_FILE, column_description=CD_FILE, has_header=True, delimiter=",")
        
        model.fit(train_pool, eval_set = val_pool)
        pickle.dump(model, open(input_dir + f'mod_on_mod_{i}.sav', 'wb'))
        
        pred = model.predict(test_pool)
        cum_y_test += pred.reshape((len(test_data),1))
        
        pred = model.predict(val_pool)
        df = pd.read_csv(VAL_FILE)
        pred_y_train[df.id] = pred.reshape((len(df),1))
        
    return (cum_y_test/K).clip(int(min(train_data.value)), int(max(train_data.value))).reshape(-1),\
        sqrt(mean_squared_error(list(train_data['value']), list(pred_y_train)))

In [12]:
def write_ans(fl_name, y_test):
    dict_ans = defaultdict(list)
    dict_ans['id'] = list(range(len(y_test)))
    dict_ans['value'] = list(y_test)
    df_ans = pd.DataFrame.from_dict(dict_ans)
    df_ans.to_csv(sub_dir+'/'+fl_name+'.csv', index=False)
    return 'Answer /%s succesfully writed' % fl_name

In [19]:
%%time

vec_qual = []

for itr, model in enumerate(Regressors):
    y_test, qual = FitOnKFolds(model)
    vec_qual.append(qual)
    write_ans(f'sub_modonmod_ctb_itr={iterations}_lr={learning_rate}_depth={depth}_fld={K}_{itr}vers', y_test)

0:	learn: 423.4403648	test: 403.2282811	best: 403.2282811 (0)	total: 26.8ms	remaining: 2m 14s
1:	learn: 414.4185225	test: 394.1187414	best: 394.1187414 (1)	total: 58.8ms	remaining: 2m 26s
2:	learn: 406.2274008	test: 385.6111748	best: 385.6111748 (2)	total: 81.9ms	remaining: 2m 16s
3:	learn: 398.6045575	test: 377.7618475	best: 377.7618475 (3)	total: 106ms	remaining: 2m 12s
4:	learn: 391.6567781	test: 370.6348536	best: 370.6348536 (4)	total: 126ms	remaining: 2m 6s
5:	learn: 385.7497972	test: 364.3009004	best: 364.3009004 (5)	total: 146ms	remaining: 2m 1s
6:	learn: 379.1845109	test: 357.6459553	best: 357.6459553 (6)	total: 170ms	remaining: 2m 1s
7:	learn: 373.2801754	test: 351.6350856	best: 351.6350856 (7)	total: 197ms	remaining: 2m 2s
8:	learn: 368.3522268	test: 346.6089582	best: 346.6089582 (8)	total: 223ms	remaining: 2m 3s
9:	learn: 364.3768590	test: 342.5415622	best: 342.5415622 (9)	total: 239ms	remaining: 1m 59s
10:	learn: 360.0342142	test: 338.2309985	best: 338.2309985 (10)	total: 2

88:	learn: 272.2553709	test: 254.3364324	best: 254.3364324 (88)	total: 2.65s	remaining: 2m 26s
89:	learn: 271.7361338	test: 254.4133227	best: 254.3364324 (88)	total: 2.69s	remaining: 2m 26s
90:	learn: 271.0511000	test: 253.6971358	best: 253.6971358 (90)	total: 2.72s	remaining: 2m 26s
91:	learn: 270.6391784	test: 253.3805343	best: 253.3805343 (91)	total: 2.75s	remaining: 2m 26s
92:	learn: 270.4735405	test: 253.2633012	best: 253.2633012 (92)	total: 2.77s	remaining: 2m 26s
93:	learn: 269.7537943	test: 252.8158719	best: 252.8158719 (93)	total: 2.8s	remaining: 2m 25s
94:	learn: 269.5091376	test: 252.6355619	best: 252.6355619 (94)	total: 2.82s	remaining: 2m 25s
95:	learn: 269.3020239	test: 252.6131487	best: 252.6131487 (95)	total: 2.85s	remaining: 2m 25s
96:	learn: 269.0877505	test: 252.5177668	best: 252.5177668 (96)	total: 2.88s	remaining: 2m 25s
97:	learn: 268.6833880	test: 252.5783096	best: 252.5177668 (96)	total: 2.91s	remaining: 2m 25s
98:	learn: 268.1456015	test: 252.2091287	best: 252.

173:	learn: 245.6006730	test: 241.0208831	best: 241.0208831 (173)	total: 5.34s	remaining: 2m 28s
174:	learn: 245.2250613	test: 240.8308759	best: 240.8308759 (174)	total: 5.37s	remaining: 2m 28s
175:	learn: 244.8629847	test: 240.7980293	best: 240.7980293 (175)	total: 5.41s	remaining: 2m 28s
176:	learn: 244.6931110	test: 240.6362436	best: 240.6362436 (176)	total: 5.44s	remaining: 2m 28s
177:	learn: 244.3070584	test: 240.5212943	best: 240.5212943 (177)	total: 5.47s	remaining: 2m 28s
178:	learn: 244.0756862	test: 240.3475376	best: 240.3475376 (178)	total: 5.5s	remaining: 2m 28s
179:	learn: 243.9852820	test: 240.2613042	best: 240.2613042 (179)	total: 5.55s	remaining: 2m 28s
180:	learn: 243.8366672	test: 240.2113864	best: 240.2113864 (180)	total: 5.59s	remaining: 2m 28s
181:	learn: 243.5347326	test: 239.9547434	best: 239.9547434 (181)	total: 5.64s	remaining: 2m 29s
182:	learn: 243.3034989	test: 239.8594816	best: 239.8594816 (182)	total: 5.68s	remaining: 2m 29s
183:	learn: 243.1836986	test: 2

261:	learn: 229.0164089	test: 232.7931379	best: 232.7931379 (261)	total: 8.46s	remaining: 2m 32s
262:	learn: 228.6659337	test: 232.7001086	best: 232.7001086 (262)	total: 8.49s	remaining: 2m 32s
263:	learn: 228.5237295	test: 232.5686311	best: 232.5686311 (263)	total: 8.52s	remaining: 2m 32s
264:	learn: 228.4103120	test: 232.5760515	best: 232.5686311 (263)	total: 8.55s	remaining: 2m 32s
265:	learn: 228.3465793	test: 232.6250368	best: 232.5686311 (263)	total: 8.59s	remaining: 2m 32s
266:	learn: 228.1345621	test: 232.3958510	best: 232.3958510 (266)	total: 8.61s	remaining: 2m 32s
267:	learn: 228.0426816	test: 232.3745406	best: 232.3745406 (267)	total: 8.64s	remaining: 2m 32s
268:	learn: 227.8793628	test: 232.2134937	best: 232.2134937 (268)	total: 8.69s	remaining: 2m 32s
269:	learn: 227.6480740	test: 232.1257295	best: 232.1257295 (269)	total: 8.72s	remaining: 2m 32s
270:	learn: 227.5301113	test: 232.0596679	best: 232.0596679 (270)	total: 8.76s	remaining: 2m 32s
271:	learn: 227.4481355	test: 

351:	learn: 216.8347582	test: 226.6385940	best: 226.6385940 (351)	total: 11.4s	remaining: 2m 30s
352:	learn: 216.6974353	test: 226.5856703	best: 226.5856703 (352)	total: 11.4s	remaining: 2m 30s
353:	learn: 216.5101178	test: 226.4448947	best: 226.4448947 (353)	total: 11.4s	remaining: 2m 30s
354:	learn: 216.3956364	test: 226.2887302	best: 226.2887302 (354)	total: 11.5s	remaining: 2m 29s
355:	learn: 216.3297441	test: 226.2769547	best: 226.2769547 (355)	total: 11.5s	remaining: 2m 29s
356:	learn: 216.2984573	test: 226.2797925	best: 226.2769547 (355)	total: 11.5s	remaining: 2m 29s
357:	learn: 216.2398422	test: 226.2334388	best: 226.2334388 (357)	total: 11.5s	remaining: 2m 29s
358:	learn: 216.1342536	test: 226.0738141	best: 226.0738141 (358)	total: 11.6s	remaining: 2m 29s
359:	learn: 215.9795655	test: 226.1135263	best: 226.0738141 (358)	total: 11.6s	remaining: 2m 29s
360:	learn: 215.8439498	test: 226.0663452	best: 226.0663452 (360)	total: 11.7s	remaining: 2m 29s
361:	learn: 215.7105246	test: 

...

4592:	learn: 126.0495806	test: 217.9918359	best: 217.9813827 (4577)	total: 2m 25s	remaining: 12.9s
4593:	learn: 126.0492868	test: 217.9915824	best: 217.9813827 (4577)	total: 2m 25s	remaining: 12.9s
4594:	learn: 126.0274209	test: 217.9872675	best: 217.9813827 (4577)	total: 2m 25s	remaining: 12.8s
4595:	learn: 126.0176387	test: 217.9820116	best: 217.9813827 (4577)	total: 2m 25s	remaining: 12.8s
4596:	learn: 126.0168233	test: 217.9827698	best: 217.9813827 (4577)	total: 2m 25s	remaining: 12.8s
4597:	learn: 126.0149998	test: 217.9854390	best: 217.9813827 (4577)	total: 2m 25s	remaining: 12.7s
4598:	learn: 126.0086603	test: 217.9832797	best: 217.9813827 (4577)	total: 2m 25s	remaining: 12.7s
4599:	learn: 126.0076490	test: 217.9830145	best: 217.9813827 (4577)	total: 2m 25s	remaining: 12.7s
4600:	learn: 125.9932254	test: 217.9755904	best: 217.9755904 (4600)	total: 2m 25s	remaining: 12.6s
4601:	learn: 125.9856866	test: 217.9631597	best: 217.9631597 (4601)	total: 2m 25s	remaining: 12.6s
4602:	lear

4679:	learn: 125.4125898	test: 217.8187551	best: 217.8081988 (4671)	total: 2m 28s	remaining: 10.1s
4680:	learn: 125.4061262	test: 217.8196354	best: 217.8081988 (4671)	total: 2m 28s	remaining: 10.1s
4681:	learn: 125.4008985	test: 217.8235142	best: 217.8081988 (4671)	total: 2m 28s	remaining: 10.1s
4682:	learn: 125.3989240	test: 217.8238969	best: 217.8081988 (4671)	total: 2m 28s	remaining: 10s
4683:	learn: 125.3969377	test: 217.8192683	best: 217.8081988 (4671)	total: 2m 28s	remaining: 10s
4684:	learn: 125.3571787	test: 217.8045205	best: 217.8045205 (4684)	total: 2m 28s	remaining: 9.97s
4685:	learn: 125.3499534	test: 217.8006425	best: 217.8006425 (4685)	total: 2m 28s	remaining: 9.94s
4686:	learn: 125.3349250	test: 217.8050144	best: 217.8006425 (4685)	total: 2m 28s	remaining: 9.91s
4687:	learn: 125.3328253	test: 217.8063382	best: 217.8006425 (4685)	total: 2m 28s	remaining: 9.88s
4688:	learn: 125.3317357	test: 217.8066515	best: 217.8006425 (4685)	total: 2m 28s	remaining: 9.84s
4689:	learn: 1

4764:	learn: 124.6928751	test: 217.6966727	best: 217.6947971 (4761)	total: 2m 31s	remaining: 7.45s
4765:	learn: 124.6924181	test: 217.6956024	best: 217.6947971 (4761)	total: 2m 31s	remaining: 7.42s
4766:	learn: 124.6812221	test: 217.7037167	best: 217.6947971 (4761)	total: 2m 31s	remaining: 7.39s
4767:	learn: 124.6541687	test: 217.6899519	best: 217.6899519 (4767)	total: 2m 31s	remaining: 7.35s
4768:	learn: 124.6491738	test: 217.6874703	best: 217.6874703 (4768)	total: 2m 31s	remaining: 7.32s
4769:	learn: 124.6333108	test: 217.6828728	best: 217.6828728 (4769)	total: 2m 31s	remaining: 7.29s
4770:	learn: 124.6253257	test: 217.6772406	best: 217.6772406 (4770)	total: 2m 31s	remaining: 7.26s
4771:	learn: 124.6241389	test: 217.6780211	best: 217.6772406 (4770)	total: 2m 31s	remaining: 7.23s
4772:	learn: 124.6173912	test: 217.6759436	best: 217.6759436 (4772)	total: 2m 31s	remaining: 7.2s
4773:	learn: 124.6034237	test: 217.6703165	best: 217.6703165 (4773)	total: 2m 31s	remaining: 7.17s
4774:	learn

4850:	learn: 124.1514949	test: 217.6283558	best: 217.6151872 (4836)	total: 2m 34s	remaining: 4.73s
4851:	learn: 124.1467400	test: 217.6231561	best: 217.6151872 (4836)	total: 2m 34s	remaining: 4.7s
4852:	learn: 124.1466248	test: 217.6226800	best: 217.6151872 (4836)	total: 2m 34s	remaining: 4.67s
4853:	learn: 124.1425403	test: 217.6240951	best: 217.6151872 (4836)	total: 2m 34s	remaining: 4.64s
4854:	learn: 124.1368113	test: 217.6051539	best: 217.6051539 (4854)	total: 2m 34s	remaining: 4.61s
4855:	learn: 124.1234420	test: 217.6044697	best: 217.6044697 (4855)	total: 2m 34s	remaining: 4.58s
4856:	learn: 124.1231365	test: 217.6051874	best: 217.6044697 (4855)	total: 2m 34s	remaining: 4.54s
4857:	learn: 124.1167052	test: 217.6012611	best: 217.6012611 (4857)	total: 2m 34s	remaining: 4.51s
4858:	learn: 124.1144456	test: 217.6001025	best: 217.6001025 (4858)	total: 2m 34s	remaining: 4.48s
4859:	learn: 124.1140101	test: 217.5991208	best: 217.5991208 (4859)	total: 2m 34s	remaining: 4.45s
4860:	learn

4933:	learn: 123.5622745	test: 217.5268744	best: 217.5268744 (4933)	total: 2m 36s	remaining: 2.1s
4934:	learn: 123.5534850	test: 217.5155103	best: 217.5155103 (4934)	total: 2m 36s	remaining: 2.07s
4935:	learn: 123.5490488	test: 217.5180584	best: 217.5155103 (4934)	total: 2m 36s	remaining: 2.04s
4936:	learn: 123.5433888	test: 217.5176573	best: 217.5155103 (4934)	total: 2m 37s	remaining: 2s
4937:	learn: 123.5226956	test: 217.5117369	best: 217.5117369 (4937)	total: 2m 37s	remaining: 1.97s
4938:	learn: 123.5133235	test: 217.5079802	best: 217.5079802 (4938)	total: 2m 37s	remaining: 1.94s
4939:	learn: 123.5111160	test: 217.5043624	best: 217.5043624 (4939)	total: 2m 37s	remaining: 1.91s
4940:	learn: 123.5095838	test: 217.5034433	best: 217.5034433 (4940)	total: 2m 37s	remaining: 1.88s
4941:	learn: 123.5068495	test: 217.5019162	best: 217.5019162 (4941)	total: 2m 37s	remaining: 1.84s
4942:	learn: 123.5056118	test: 217.5009741	best: 217.5009741 (4942)	total: 2m 37s	remaining: 1.81s
4943:	learn: 1

In [ ]:
print(vec_qual)
#model = pickle.load(open(input_dir + f'mod_part_feat_{i}.sav', 'rb'))

In [27]:
y_test

array([8.11094601e+02, 1.25944387e+03, 1.88162684e+02, 6.45633283e+02,
       1.87585182e+01, 2.54373734e+02, 2.89414607e+02, 2.17975750e+02,
       3.55959822e+02, 3.46470991e+02, 5.29974450e+01, 4.81745132e+02,
       2.09402193e+01, 1.21336378e+02, 8.44906707e+01, 3.57669972e+02,
       1.12501530e+03, 3.30770002e+01, 2.50034467e+01, 1.42314889e+02,
       1.56357059e+02, 6.31172944e+02, 1.07757025e+02, 2.73395793e+02,
       3.64866183e+02, 1.67088656e+02, 2.73903208e+02, 3.39717351e+02,
       1.57970950e+02, 5.74300674e+02, 3.61303298e+01, 1.53865856e+02,
       2.61720221e+02, 2.02305947e+02, 1.42428089e+02, 0.00000000e+00,
       2.04919958e+02, 5.42722028e+02, 2.32212582e+02, 1.39027216e+02,
       3.93185223e+01, 3.17697914e+01, 4.37190971e+02, 2.87290816e+02,
       1.27067846e+02, 1.04293395e+03, 6.04582700e+02, 1.67182486e+03,
       4.96718897e+00, 3.48855793e+02, 1.00256621e+02, 2.72814935e+02,
       0.00000000e+00, 4.01857241e+02, 1.97480456e+02, 2.06753302e+02,
      

In [28]:
test_data_may = test_data

In [29]:
test_data_may['value'] = list(y_test)

In [30]:
test_data_may

,id,bulk_id,spalen,date1,start_square,value,price,mean_sq,mean_fl,month,month_cnt,Класс объекта,Количество помещений,Огорожена территория,Площадь земельного участка,Входные группы,Детский сад,Школа,Поликлиника,ФОК,Автомойка,Кладовые,Колясочные,Кондиционирование,Вентлияция,Видеонаблюдение,Подземная парковка,Двор без машин,Машиномест,Площадь пром. зоны в радиусе 500 м,Площадь зеленой зоны в радиусе 500 м,До Кремля,До ТТК(км),До Садового(км),До большой дороги на машине(км),До удобной авторазвязки на машине(км),До метро пешком(км),До промки(км),До парка(км),До парка пешком(км),Станций метро от кольца,Площадь двора,Курс,Cтавка по ипотеке,Вклады до 1 года,Вклады от 1 года до 3 лет,Вклады свыше 3 лет,plan
0,0,FF3814A9-9F7D-E711-8530-00505688958B,1,2018-03-01,8547.70,811.094601,147387.111111,37.99,9,3,39,эконом,1983,нет,17.0,да,345,825,0,0,нет,да,да,1,1,0,да,да,1800,65357,25533,18.0,12.0,16.0,0.55,1.70,1.40,0.28,0.58,0.93,8,7640,57.063605,9.7,5.53,6.13,5.63,224
1,1,FF3814A9-9F7D-E711-8530-00505688958B,2,2018-03-01,13471.70,1259.443867,138465.811966,57.80,9,3,39,эконом,1983,нет,17.0,да,345,825,0,0,нет,да,да,1,1,0,да,да,1800,65357,25533,18.0,12.0,16.0,0.55,1.70,1.40,0.28,0.58,0.93,8,7640,57.063605,9.7,5.53,6.13,5.63,233
2,2,FF3814A9-9F7D-E711-8530-00505688958B,0,2018-03-01,1136.00,188.162684,171589.583333,23.67,10,3,39,эконом,1983,нет,17.0,да,345,825,0,0,нет,да,да,1,1,0,да,да,1800,65357,25533,18.0,12.0,16.0,0.55,1.70,1.40,0.28,0.58,0.93,8,7640,57.063605,9.7,5.53,6.13,5.63,47
3,3,FF3814A9-9F7D-E711-8530-00505688958B,3,2018-03-01,6598.00,645.633283,134381.707317,80.46,8,3,39,эконом,1983,нет,17.0,да,345,825,0,0,нет,да,да,1,1,0,да,да,1800,65357,25533,18.0,12.0,16.0,0.55,1.70,1.40,0.28,0.58,0.93,8,7640,57.063605,9.7,5.53,6.13,5.63,82
4,20,7726CF5A-CC0A-E511-8098-001EC9D56418,3,2018-03-01,187.20,18.758518,182700.000000,93.60,3,3,39,комфорт,1094,нет,6.8,да,150,550,0,1,нет,да,да,1,1,1,да,да,871,4740,156205,13.0,8.0,11.0,0.69,7.00,0.35,0.45,0.40,0.54,4,8587,57.063605,9.7,5.53,6.13,5.63,2
5,23,72B1CAF6-A2DC-E511-9840-001EC9D56418,2,2018-03-01,577.80,254.373734,110470.000000,57.78,9,3,39,комфорт,3958,нет,114.0,да,800,3150,600,1,нет,да,да,1,2,1,да,да,6000,13990,7000,20.0,14.0,17.0,1.80,6.70,5.80,0.23,1.20,1.60,7,7133,57.063605,9.7,5.53,6.13,5.63,9
6,24,35168781-8F60-E611-9523-001EC9D5643C,3,2018-03-01,5052.10,289.414607,89142.592593,93.56,8,3,39,комфорт,3958,нет,114.0,да,800,3150,600,1,нет,да,да,1,2,1,да,да,6000,13990,0,20.0,14.0,17.0,2.10,7.00,6.00,0.23,1.20,1.40,7,7133,57.063605,9.7,5.53,6.13,5.63,53
7,25,35168781-8F60-E611-9523-001EC9D5643C,2,2018-03-01,301.50,217.975750,100566.666667,61.92,10,3,39,комфорт,3958,нет,114.0,да,800,3150,600,1,нет,да,да,1,2,1,да,да,6000,13990,0,20.0,14.0,17.0,2.10,7.00,6.00,0.23,1.20,1.40,7,7133,57.063605,9.7,5.53,6.13,5.63,4
8,26,A8632090-8F60-E611-9523-001EC9D5643C,3,2018-03-01,2626.60,355.959822,88275.862069,90.57,12,3,39,комфорт,3958,нет,114.0,да,800,3150,600,1,нет,да,да,1,2,1,да,да,6000,13990,0,20.0,14.0,17.0,2.30,7.20,6.00,0.23,1.20,1.30,7,7133,57.063605,9.7,5.53,6.13,5.63,29
9,27,A8632090-8F60-E611-9523-001EC9D5643C,2,2018-03-01,530.10,346.470991,104522.222222,58.90,17,3,39,комфорт,3958,нет,114.0,да,800,3150,600,1,нет,да,да,1,2,1,да,да,6000,13990,0,20.0,14.0,17.0,2.30,7.20,6.00,0.23,1.20,1.30,7,7133,57.063605,9.7,5.53,6.13,5.63,9


#### Unload Data

In [31]:
test_data_may.to_csv(sub_dir + f"/subsub_full_feat_may.csv")